In [2]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import joblib
import pygad
import numpy as np
import pyswarms as ps
from neat import Config, Population, nn, statistics
import neat
import pickle
from pureples.shared.substrate import Substrate
from pureples.es_hyperneat.es_hyperneat import ESNetwork
from pureples.hyperneat.hyperneat import create_phenotype_network

## Лабораторная работа 2

### Задание 1

In [3]:
def generate_dataset(num_samples):
    data = []
    for _ in range(num_samples):
        object1_state = [
            random.choice([0, 1]),
            random.choice(['Сфера', 'Куб', 'Пирамида']),  # Номинальный признак
            random.choice(['Вперед', 'Назад', 'Лево', 'Право']),  # Порядковый признак
            random.uniform(0, 360)  # Количественный признак
        ]
        
        # Состояние объекта 2
        object2_state = [
            random.choice([0, 1]),  # Бинарный признак
            random.choice(['Сфера', 'Куб', 'Пирамида']),  # Номинальный признак
            random.choice(['Вперед', 'Назад', 'Лево', 'Право']),  # Порядковый признак
            random.uniform(0, 360)  # Количественный признак
        ]
        
        collision = random.choice(['Да', 'Нет'])
        
        data.append(object1_state + object2_state + [collision])
    
    columns = [
        'Объект1_Бинарный', 'Объект1_Форма', 'Объект1_Направление', 'Объект1_Угол',
        'Объект2_Бинарный', 'Объект2_Форма', 'Объект2_Направление', 'Объект2_Угол',
        'Коллизия'
    ]
    return pd.DataFrame(data, columns=columns)

datasets = []
sample_ranges = [(30, 100), (100, 500), (500, 1000), (1000, 2000)]
feature_ranges = ['4-7', '8-10', '10+']

for sample_range in sample_ranges:
    for _ in range(3):
        num_samples = random.randint(*sample_range)
        dataset = generate_dataset(num_samples)
        datasets.append(dataset)

for i, dataset in enumerate(datasets):
    dataset.to_csv(f'dataset_{i+1}.csv', index=False)

### Задание 2

In [4]:
df = datasets[0]

X = df.drop('Коллизия', axis=1)
y = df['Коллизия']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['Объект1_Угол', 'Объект2_Угол']
numeric_transformer = StandardScaler()

categorical_features = ['Объект1_Форма', 'Объект1_Направление', 'Объект2_Форма', 'Объект2_Направление']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

binary_features = ['Объект1_Бинарный', 'Объект2_Бинарный']
binary_transformer = 'passthrough'

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_features)
    ])

models = {
    # Логистическая регрессия - быстрый и интерпретируемый алгоритм
    'Logistic Regression': LogisticRegression(max_iter=1000),
    # Решающее дерево - быстрый и не требует масштабирования данных
    'Decision Tree': DecisionTreeClassifier(),
    # Случайный лес - устойчив к переобучению, хорош для разнородных данных
    'Random Forest': RandomForestClassifier(),
    # SVM - хорош для сложных границ решений, но требует больше ресурсов
    'SVM': SVC(probability=True),
    # Наивный Байес - очень быстрый, хорошо работает с категориальными данными
    'Naive Bayes': GaussianNB()
}

results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    joblib.dump(pipeline, f'{name.replace(" ", "_")}_v2_model.pkl')
    print(f'{name}: Accuracy = {accuracy:.4f}')

print("\nРезультаты точности моделей:")
for name, accuracy in results.items():
    print(f"{name}: {accuracy:.4f}")

Logistic Regression: Accuracy = 0.5000
Decision Tree: Accuracy = 0.5000
Random Forest: Accuracy = 0.3889
SVM: Accuracy = 0.5000
Naive Bayes: Accuracy = 0.3889

Результаты точности моделей:
Logistic Regression: 0.5000
Decision Tree: 0.5000
Random Forest: 0.3889
SVM: 0.5000
Naive Bayes: 0.3889


In [5]:
best_model_name = max(results, key=results.get)
best_model_accuracy = results[best_model_name]
print(f"\nЛучшая модель: {best_model_name} с точностью {best_model_accuracy:.4f}")

loaded_model = joblib.load(f'{best_model_name.replace(" ", "_")}_v2_model.pkl')
sample_data = X_test.iloc[[0]]
prediction = loaded_model.predict(sample_data)
print(f"\nПример предсказания для строки:\n{sample_data}\nПредсказание: {prediction[0]}")
print(f"Реальное значение: {y_test.iloc[0]}")


Лучшая модель: Logistic Regression с точностью 0.5000

Пример предсказания для строки:
    Объект1_Бинарный Объект1_Форма Объект1_Направление  Объект1_Угол  \
44                 0      Пирамида               Назад    128.604488   

    Объект2_Бинарный Объект2_Форма Объект2_Направление  Объект2_Угол  
44                 0         Сфера              Вперед    308.682133  
Предсказание: Нет
Реальное значение: Нет


## Лабораторная работа 3

### PyGAD

In [6]:
def optimize_with_pygad(model_name, X_train, y_train, X_test, y_test):
    # Определение пространства параметров
    if model_name == 'SVM':
        gene_space = [
            {'low': 0.1, 'high': 10},   # C
            {'low': 0, 'high': 1},       # kernel (0-linear, 1-rbf)
            {'low': 0, 'high': 1}        # gamma (0-scale, 1-auto)
        ]
    elif model_name == 'Naive Bayes':
        gene_space = [{'low': 1e-10, 'high': 1e-1}]  # var_smoothing

    # Функция приспособленности
    def fitness_func(ga_instance, solution, solution_idx):
        if model_name == 'SVM':
            params = {
                'C': solution[0],
                'kernel': ['linear', 'rbf'][int(solution[1])],
                'gamma': ['scale', 'auto'][int(solution[2])]
            }
            model = SVC(**params)
        elif model_name == 'Naive Bayes':
            params = {'var_smoothing': solution[0]}
            model = GaussianNB(**params)
        
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy

    # Настройка алгоритма
    ga = pygad.GA(
        num_generations=20,
        num_parents_mating=3,
        sol_per_pop=10,
        num_genes=len(gene_space),
        gene_space=gene_space,
        fitness_func=fitness_func,
        mutation_num_genes=1
    )
    ga.run()
    
    # Сохранение лучшей модели
    best_params = ga.best_solution()[0]
    if model_name == 'SVM':
        best_model = SVC(C=best_params[0], 
                        kernel=['linear', 'rbf'][int(best_params[1])], 
                        gamma=['scale', 'auto'][int(best_params[2])])
    else:
        best_model = GaussianNB(var_smoothing=best_params[0])
    
    best_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', best_model)
    ])
    best_pipeline.fit(X_train, y_train)
    joblib.dump(best_pipeline, f'{model_name}_pygad_optimized.pkl')
    return best_params

# Оптимизация SVM
best_svm_params = optimize_with_pygad('SVM', X_train, y_train, X_test, y_test)
print(f"Лучшие параметры SVM: {best_svm_params}")

# Оптимизация Naive Bayes
best_nb_params = optimize_with_pygad('Naive Bayes', X_train, y_train, X_test, y_test)
print(f"Лучшие параметры Naive Bayes: {best_nb_params}")

Лучшие параметры SVM: [0.10452044 0.89589591 0.42568329]
Лучшие параметры Naive Bayes: [0.07943226]


### PySwarms

In [7]:
def optimize_with_pyswarms(model_name, X_train, y_train, X_test, y_test):
    # Пространство поиска
    if model_name == 'SVM':
        bounds = ([0.1, 0, 0], [10, 1, 1])  # C, kernel, gamma
    else:
        bounds = ([1e-10], [1e-1])          # var_smoothing

    # Функция стоимости
    def f(x):
        scores = []
        for params in x:
            try:
                if model_name == 'SVM':
                    model = SVC(
                        C=params[0],
                        kernel=['linear', 'rbf'][int(params[1])],
                        gamma=['scale', 'auto'][int(params[2])]
                    )
                else:
                    model = GaussianNB(var_smoothing=params[0])
                
                pipeline = Pipeline([
                    ('preprocessor', preprocessor),
                    ('classifier', model)
                ])
                pipeline.fit(X_train, y_train)
                y_pred = pipeline.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                scores.append(1 - accuracy)  # Минимизируем ошибку
            except:
                scores.append(1)  # Штраф за невалидные параметры
        return np.array(scores)

    # Запуск оптимизации
    optimizer = ps.single.GlobalBestPSO(
        n_particles=20,
        dimensions=len(bounds[0]),
        options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
        bounds=bounds
    )
    best_cost, best_pos = optimizer.optimize(f, iters=50)
    
    # Сохранение модели
    if model_name == 'SVM':
        best_model = SVC(
            C=best_pos[0],
            kernel=['linear', 'rbf'][int(best_pos[1])],
            gamma=['scale', 'auto'][int(best_pos[2])]
        )
    else:
        best_model = GaussianNB(var_smoothing=best_pos[0])
    
    best_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', best_model)
    ])
    best_pipeline.fit(X_train, y_train)
    joblib.dump(best_pipeline, f'{model_name}_pyswarms_optimized.pkl')
    return best_pos

# Оптимизация SVM
best_swarm_svm = optimize_with_pyswarms('SVM', X_train, y_train, X_test, y_test)
print(f"Лучшие параметры SVM (PSO): {best_swarm_svm}")

2025-05-24 07:05:07,099 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|50/50, best_cost=0.5
2025-05-24 07:05:15,700 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.5, best pos: [0.69053755 0.38016987 0.79807297]


Лучшие параметры SVM (PSO): [0.69053755 0.38016987 0.79807297]


### NEAT

In [8]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = 0
        X_transformed = preprocessor.transform(X_train)
        
        for i in range(len(X_transformed)):
            inputs = X_transformed[i]
            output = net.activate(inputs)
            pred = 'Да' if output[0] > 0.5 else 'Нет'
            genome.fitness += 1 if pred == y_train.iloc[i] else 0
            
        genome.fitness /= len(X_transformed)

def run_neat(dataset):
    config = Config(neat.DefaultGenome,
                    neat.DefaultReproduction,
                    neat.DefaultSpeciesSet,
                    neat.DefaultStagnation,
                    'neat_config.txt')
    
    pop = Population(config)
    pop.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    
    winner = pop.run(eval_genomes, 50)
    
    # Сохранение лучшей сети
    with open('best_neat.pkl', 'wb') as f:
        pickle.dump(winner, f)
    
    return winner

# Загрузка минимального и максимального датасетов
min_dataset = datasets[0]  # Самый маленький
max_dataset = datasets[-1] # Самый большой

# Запуск NEAT для обоих датасетов
neat_winner_min = run_neat(min_dataset)
neat_winner_max = run_neat(max_dataset)


 ****** Running generation 0 ****** 

Population's average fitness: 0.49822 stdev: 0.05758
Best fitness: 0.64789 - size: (1, 18) - species 1 - id 131
Average adjusted fitness: 0.160
Mean genetic distance 1.186, standard deviation 0.356
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      0.6    0.160     0
Total extinctions: 0
Generation time: 0.446 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.54235 stdev: 0.05173
Best fitness: 0.70423 - size: (1, 17) - species 1 - id 266
Average adjusted fitness: 0.120
Mean genetic distance 1.177, standard deviation 0.400
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      0.7    0.120     0
Total extinctions: 0
Generation time: 0.460 sec (0.453 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.57249 stdev